<a href="https://colab.research.google.com/github/jdmooreno211/API_DetectBluekeep/blob/master/Consultas%20Graficas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install mysql-connector-python

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.3/19.3 MB 54.5 MB/s eta 0:00:00


In [29]:
import mysql.connector
import pandas as pd
import plotly.express as px
from mysql.connector import Error

# Datos de conexión
host_name = "3.145.203.151"
user_name = "biucundi"
user_password = "0y3m14m0r"
db_name = "Chinook"

# Función para ejecutar consultas y devolver resultados en un DataFrame
def execute_query(query):
    try:
        connection = mysql.connector.connect(
            host=host_name,
            user=user_name,
            password=user_password,
            database=db_name
        )
        if connection.is_connected():
            cursor = connection.cursor()
            cursor.execute(query)
            results = cursor.fetchall()
            columns = [desc[0] for desc in cursor.description]
            return pd.DataFrame(results, columns=columns)
    except Error as e:
        print(f"Error al conectar a la base de datos: {e}")
    finally:
        if connection.is_connected():
            cursor.close()
            connection.close()

# 1. Top 10 artistas con más ventas
query1 = """
SELECT Artist.Name AS ArtistName, SUM(InvoiceLine.Quantity) AS TotalUnitsSold
FROM Artist
JOIN Album ON Artist.ArtistId = Album.ArtistId
JOIN Track ON Album.AlbumId = Track.AlbumId
JOIN InvoiceLine ON Track.TrackId = InvoiceLine.TrackId
GROUP BY Artist.Name
ORDER BY TotalUnitsSold DESC
LIMIT 10;
"""
df1 = execute_query(query1)
fig1 = px.bar(df1, x='TotalUnitsSold', y='ArtistName', orientation='h',
             title='Top 10 artistas con más ventas', labels={'TotalUnitsSold':'Unidades vendidas', 'ArtistName':'Artista'})
fig1.show()

# 2. Ventas por país
query2 = """
SELECT BillingCountry AS Country, SUM(Total) AS TotalSales
FROM Invoice
GROUP BY BillingCountry
ORDER BY TotalSales DESC;
"""
df2 = execute_query(query2)
fig2 = px.choropleth(df2, locations='Country', locationmode='country names', color='TotalSales',
                    title='Ventas por país', color_continuous_scale=px.colors.sequential.Plasma)
fig2.show()

# 3. Distribución de métodos de pago
query3 = """
SELECT Invoice.BillingPostalCode AS PostalCode, COUNT(*) AS PaymentCount
FROM Invoice
GROUP BY Invoice.BillingPostalCode;
"""
df3 = execute_query(query3)
fig3 = px.pie(df3, names='PostalCode', values='PaymentCount',
             title='Distribución de métodos de pago por código postal')
fig3.show()

# 4. Total de ventas por género musical
query4 = """
SELECT Genre.Name AS Genre, SUM(InvoiceLine.Quantity) AS TotalUnitsSold
FROM Genre
JOIN Track ON Genre.GenreId = Track.GenreId
JOIN InvoiceLine ON Track.TrackId = InvoiceLine.TrackId
GROUP BY Genre.Name
ORDER BY TotalUnitsSold DESC;
"""
df4 = execute_query(query4)
fig4 = px.bar(df4, x='Genre', y='TotalUnitsSold', title='Ventas por género musical',
             labels={'TotalUnitsSold':'Unidades vendidas', 'Genre':'Género'})
fig4.show()

# 5. Ventas mensuales totales
query5 = """
SELECT DATE_FORMAT(Invoice.InvoiceDate, '%Y-%m') AS Month, SUM(Invoice.Total) AS TotalSales
FROM Invoice
GROUP BY Month
ORDER BY Month;
"""
df5 = execute_query(query5)
fig5 = px.line(df5, x='Month', y='TotalSales', title='Tendencia de ventas mensuales',
              labels={'Month':'Mes', 'TotalSales':'Ventas totales'})
fig5.show()
